In [25]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow
from keras.preprocessing import image
import os
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
import cv2
from sklearn.preprocessing import LabelEncoder
import sys
import pickle

In [2]:
categories = ['with_mask','without_mask']

In [3]:
path = os.path.join(os.getcwd(), f'dataset\{categories[0]}')
print(os.path.exists(path))


True


In [4]:
dataset = []

for category in categories:
    path = os.path.join(os.getcwd(),f'dataset\{category}')
    
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        try:
            image = cv2.imread(img_path)
            image = cv2.resize(image, (224,224))
        #image = cv2.resize(image,(224,224))
            dataset.append([image, category])
        except:
            pass
        

In [5]:
type(dataset)

list

In [6]:
# here the dataset will be in the order, with_mask then without mask so we will shuffle the data
random.shuffle(dataset)

In [7]:
X = []
y = []
for feature,label in dataset:
    X.append(feature)
    y.append(label)

In [8]:
X = np.array(X)
y = np.array(y)

In [9]:
print(X.shape, y.shape)

(4072, 224, 224, 3) (4072,)


In [10]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
print(y)

[1 0 0 ... 1 0 1]


In [11]:
# scale the value of X i.e the features
X = X/255

In [12]:
print(X)

[[[[0.3254902  0.34901961 0.53333333]
   [0.31372549 0.33333333 0.51372549]
   [0.29803922 0.31764706 0.49019608]
   ...
   [0.05490196 0.04705882 0.12156863]
   [0.05882353 0.05098039 0.12156863]
   [0.0627451  0.05882353 0.12156863]]

  [[0.31764706 0.3372549  0.52156863]
   [0.30588235 0.32156863 0.50196078]
   [0.28627451 0.30196078 0.47843137]
   ...
   [0.05882353 0.05098039 0.1254902 ]
   [0.0627451  0.05490196 0.1254902 ]
   [0.06666667 0.05882353 0.1254902 ]]

  [[0.30588235 0.31764706 0.50588235]
   [0.29019608 0.30196078 0.48235294]
   [0.27058824 0.28235294 0.45490196]
   ...
   [0.0627451  0.05490196 0.12941176]
   [0.06666667 0.05882353 0.13333333]
   [0.07058824 0.0627451  0.13333333]]

  ...

  [[0.20784314 0.34509804 0.63529412]
   [0.19607843 0.33333333 0.61960784]
   [0.18039216 0.31764706 0.60392157]
   ...
   [0.31372549 0.31372549 0.49019608]
   [0.30196078 0.30980392 0.4745098 ]
   [0.29803922 0.30588235 0.46666667]]

  [[0.2        0.3372549  0.62352941]
   [0.1

In [13]:
X_train, X_test, y_train, y_test = tts(X, y, stratify = y, test_size = 0.2)
print(X_train.shape, " ", y_train.shape)
print(X_test.shape, " ", y_test.shape)

(3257, 224, 224, 3)   (3257,)
(815, 224, 224, 3)   (815,)


In [14]:
from keras.applications.vgg16 import VGG16

In [15]:
vgg = VGG16()

In [16]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
# in transfer learning we will remove the last layer of the predictions
# vgg is functional model but we want it to just predict 
from keras import Sequential
model = Sequential()

for layer in vgg.layers[:-1]:
    model.add(layer)
    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [18]:
# freeze the parameters of the model so that it will not change while training of the model
for layer in model.layers:
    layer.trainable = False
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [19]:
from keras.layers import Dense
model.add(Dense(1,activation='sigmoid'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [21]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [22]:
# model.fit(X_train, y_train, epochs = 5, batch_size = 32, validation_data = (X_test, y_test))

Epoch 1/5
102/102 [==============================] - 675s 7s/step - loss: 0.4978 - accuracy: 0.7777 - val_loss: 0.4389 - val_accuracy: 0.7890
Epoch 2/5
102/102 [==============================] - 627s 6s/step - loss: 0.3312 - accuracy: 0.8732 - val_loss: 0.3051 - val_accuracy: 0.8883
Epoch 3/5
102/102 [==============================] - 632s 6s/step - loss: 0.2720 - accuracy: 0.8990 - val_loss: 0.2461 - val_accuracy: 0.9104
Epoch 4/5
102/102 [==============================] - 633s 6s/step - loss: 0.2411 - accuracy: 0.9122 - val_loss: 0.2463 - val_accuracy: 0.9117
Epoch 5/5
102/102 [==============================] - 636s 6s/step - loss: 0.2111 - accuracy: 0.9266 - val_loss: 0.2292 - val_accuracy: 0.9153


In [26]:
with open('trained_model.pkl','wb') as f:
    pickle.dump(model, f)

In [27]:
def detect_face_mask(img):
    y_pred = model.predict([img.reshape(1,224,224,3)]) # we need to reshape the image because it expects a batch of images as its input
    return int(y_pred[0][0] <= 0.5) # 1 means the person is wearing the mask

In [46]:
def draw_label(img, text, pos, dim, bg_color):
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    
    cv2.rectangle(img, pos, (pos[0]+dim[0], pos[1]+dim[1]), bg_color, 2)
    cv2.putText(img, text, (pos[0], pos[1]-3), cv2.FONT_HERSHEY_SIMPLEX, 1, bg_color, 1,cv2.LINE_AA)

## Using the Haar model to detect face

In [56]:
import traceback

cap = cv2.VideoCapture(0)
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

try:
    while True:
        rel, frame = cap.read() # we will do the prediction on the frame
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor = 1.1,
            minNeighbors = 5,
            minSize = (30, 30),
            flags = cv2.CASCADE_SCALE_IMAGE
        )

        # draw the rectangle around the image
        for (x,y,w,h) in faces:
            #cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)

            # crop the face from the frame and pass it to the predictor
            crop_img = frame[y-2:y+h,  x-2:x+w]
            crop_img = cv2.resize(crop_img, (224,224))
            y_pred = detect_face_mask(crop_img)

            if y_pred == 1:
                draw_label(frame, "Mask Detected", (x-3,y-3), (w, h), (0,255,0))
            else:
                draw_label(frame, "No Mask Detected", (x-3,y-3),(w, h),(0,0,255))

        #img = cv2.resize(frame, (224,224))
        #if detect_face_mask(img) == 1:
        #    cv2.putText(frame, "Mask Detected", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1,cv2.LINE_AA)
        
        # Display the resulting frame
        cv2.imshow("window",frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

except Exception as e:
    traceback.print_exc()
    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 1s 658ms/step


## Using a DNN model of openCV to detect faces

In [69]:
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")
cap = cv2.VideoCapture(0)
try:
    while True:
        rel, frame = cap.read() # we will do the prediction on the frame
        (h, w) = frame.shape[:2]

        blob = cv2.dnn.blobFromImage(frame, 1.0,
        (300, 300), (104.0, 117.0, 123.0))
        net.setInput(blob)
        faces = net.forward()
        
        for i in range(faces.shape[2]):
            confidence = faces[0, 0, i, 2]
            if confidence > 0.5:
                box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x, y, x1, y1) = box.astype("int")
                #cv2.rectangle(frame, (x, y), (x1, y1), (0, 0, 255), 2)
                crop_img = frame[y-2:y1,  x-2:x1]
                crop_img = cv2.resize(crop_img, (224,224))
                y_pred = detect_face_mask(crop_img)
                if y_pred == 1:
                    cv2.putText(frame, "Mask Detected", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1,cv2.LINE_AA)
                    cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 2)
                else:
                    cv2.putText(frame, "No Mask Detected", (x ,y- 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1,cv2.LINE_AA)
                    cv2.rectangle(frame, (x, y), (x1, y1), (0, 0,255), 2)
                
        cv2.imshow("window",frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

except Exception as e:
    traceback.print_exc()
    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 1s 569ms/step


Traceback (most recent call last):
  File "C:\Users\Prarthana Dwivedi\AppData\Local\Temp\ipykernel_21664\521698183.py", line 20, in <module>
    crop_img = cv2.resize(crop_img, (224,224))
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

